In [47]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [49]:
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")
INDEX_NAME = os.getenv("INDEX_NAME")
INDEX_ID = os.getenv("INDEX_ID")
DB_USER= os.getenv("CLOUD_SQL_USER1")
DB_PASS = os.getenv("CLOUD_SQL_PASSWORD")

### Langchain VectorStores

In [4]:
from typing import List, Optional

from google.cloud import aiplatform
from langchain_google_vertexai import VectorSearchVectorStore, VectorSearchVectorStoreDatastore
from langchain_google_vertexai import VertexAIEmbeddings

model_name = "text-embedding-005"
embedding_model = VertexAIEmbeddings(model_name=model_name)


In [56]:
from langchain_google_vertexai import VertexAIEmbeddings

model_name = "text-embedding-005"
embedding_model = VertexAIEmbeddings(model_name=model_name)

single_embedding = embedding_model.embed_query("User query")
multiple_embeddings = embedding_model.embed_documents([
	"Sample text 1",
	"Sample text 2",
	"Sample text 3",
])

In [29]:
len(single_embedding)

768

In [38]:
def vector_search_create_index(
    project: str, location: str, display_name: str, gcs_uri: Optional[str] = None
) -> aiplatform.MatchingEngineIndex:
    """Create a vector search index.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name
        display_name (str): Required. The index display name
        gcs_uri (str): Optional. The Google Cloud Storage uri for index content

    Returns:
        The created MatchingEngineIndex.
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # Create Index
    index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
        display_name=display_name,
        contents_delta_uri=gcs_uri,
        description="Matching Engine Index",
        dimensions=100,
        approximate_neighbors_count=150,
        leaf_node_embedding_count=500,
        leaf_nodes_to_search_percent=7,
        index_update_method="BATCH_UPDATE",  # Options: STREAM_UPDATE, BATCH_UPDATE
        distance_measure_type=aiplatform.matching_engine.matching_engine_index_config.DistanceMeasureType.DOT_PRODUCT_DISTANCE,
    )

    return index


In [ ]:
created_vector_index = vector_search_create_index(project=PROJECT_ID,
                                                  location=REGION,
                                                  display_name="professional-doc-index")

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/998605230609/locations/us-central1/indexes/4780496237663092736/operations/4648919777111179264
MatchingEngineIndex created. Resource name: projects/998605230609/locations/us-central1/indexes/4780496237663092736
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/998605230609/locations/us-central1/indexes/4780496237663092736')


In [53]:
def vector_search_list_index(
    project: str, location: str
) -> List[aiplatform.MatchingEngineIndex]:
    """List vector search indexes.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name

    Returns:
        List of aiplatform.MatchingEngineIndex
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # List Indexes
    return aiplatform.MatchingEngineIndex.list()


In [65]:
vector_search_list_index(project=PROJECT_ID, location=REGION)

 resource name: projects/998605230609/locations/us-central1/indexes/4780496237663092736]

In [61]:
def vector_search_create_index_endpoint(
    project: str, location: str, display_name: str
) -> None:
    """Create a vector search index endpoint.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name
        display_name (str): Required. The index endpoint display name
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # Create Index Endpoint
    index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
        display_name=display_name,
        public_endpoint_enabled=True,
        description="Matching Engine Index Endpoint",
    )

    print(index_endpoint.name)
    return index_endpoint.name

In [62]:
created_endpoint_name = vector_search_create_index_endpoint(project=PROJECT_ID, location=REGION, display_name="professional-doc-index-endpoint")

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048/operations/9194740650988273664
MatchingEngineIndexEndpoint created. Resource name: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048')
8180749140699906048


In [63]:
created_endpoint_name

'8180749140699906048'

In [ ]:
def vector_search_deploy_index(
    project: str,
    location: str,
    index_name: str,
    index_endpoint_name: str,
    deployed_index_id: str,
) -> None:
    """Deploy a vector search index to a vector search index endpoint.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name
        index_name (str): Required. The index to update. A fully-qualified index
          resource name or a index ID.  Example:
          "projects/123/locations/us-central1/indexes/my_index_id" or
          "my_index_id".
        index_endpoint_name (str): Required. Index endpoint to deploy the index
          to.
        deployed_index_id (str): Required. The user specified ID of the
          DeployedIndex.
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # Create the index instance from an existing index
    index = aiplatform.MatchingEngineIndex(index_name=index_name)

    # Create the index endpoint instance from an existing endpoint.
    index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
        index_endpoint_name=index_endpoint_name
    )

    # Deploy Index to Endpoint
    index_endpoint = index_endpoint.deploy_index(
        index=index, deployed_index_id=deployed_index_id
    )

    print(index_endpoint.deployed_indexes)

In [73]:
INDEX_ID, created_endpoint_name

('projects/998605230609/locations/us-central1/indexes/4780496237663092736',
 '8180749140699906048')

In [75]:
vector_search_deploy_index(project=PROJECT_ID, 
                           location=REGION, 
                           index_name=INDEX_ID, 
                           index_endpoint_name=created_endpoint_name, 
                           deployed_index_id="professional_docindex_endpoint")

222222222222222222222
33333333333333333333333
Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048/operations/9072017561142427648
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048
[id: "professional_docindex_endpoint"
index: "projects/998605230609/locations/us-central1/indexes/4780496237663092736"
create_time {
  seconds: 1739812894
  nanos: 17556000
}
index_sync_time {
  seconds: 1739814439
  nanos: 622997000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]


In [ ]:
def vector_search_list_index_endpoint(
    project: str, location: str
) -> List[aiplatform.MatchingEngineIndexEndpoint]:
    """List vector search index endpoints.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name

    Returns:
        List of aiplatform.MatchingEngineIndexEndpoint
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # List Index Endpoints
    return aiplatform.MatchingEngineIndexEndpoint.list()

In [78]:
vector_search_list_index_endpoint(project=PROJECT_ID, location=REGION)

 resource name: projects/998605230609/locations/us-central1/indexEndpoints/8180749140699906048,
 resource name: projects/998605230609/locations/us-central1/indexEndpoints/9176150211465052160]

### Cloud SQL

In [1]:
from langchain_google_cloud_sql_pg import PostgresEngine

In [51]:
engine = PostgresEngine.from_instance(project_id=PROJECT_ID,
                                      region=REGION,
                                      user=DB_USER,
                                      password=DB_PASS,
                                      instance="langchain-sql-house1",
                                      database="ragdb")

In [ ]:
from langchain_google_cloud_sql_pg import Column
engine.init_vectorstore_table(table_name="professional_docs",
                              vector_size=768,
                              metadata_columns=[Column("title", "TEXT")])

In [65]:
from langchain_google_cloud_sql_pg import PostgresVectorStore
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(model_name=model_name)
store = PostgresVectorStore.create(engine=engine,
                            table_name="professional_docs",
                            embedding_service=embedding
                            )

/tmp/ipykernel_360870/3384836738.py:5: RuntimeWarning: coroutine 'PostgresVectorStore.create' was never awaited
  store = PostgresVectorStore.create(engine=engine,
